# Build a Chatbot That Acts Like a Person

In [ ]:
# create chat Bot

from dotenv import load_dotenv
import os
from openai import OpenAI
from IPython.display import Markdown,display
from pypdf import PdfReader
import gradio as gr # For creating web apps easily


In [10]:
# Read pdf file
pdfReader = PdfReader("../Resources/Profile.pdf")
prof_summary=""
for page in pdfReader.pages:
    text=page.extract_text()
    if text:
        prof_summary+=text

# print(prof_summary)


In [11]:
# Read summary file 
summ_filepath="../Resources/Summary.txt"
with open(summ_filepath,'r') as file:
    summary=file.read()
# print(summary)

In [13]:
# Load env variables
load_dotenv()

client = OpenAI(api_key=os.getenv('OPEN_API_KEY'))

print("Using API key:", os.getenv("OPEN_API_KEY")[:10] + "..." if os.getenv("OPEN_API_KEY") else "No key found!")
openaiclient = OpenAI(api_key=os.getenv('OPEN_API_KEY'))



Using API key: sk-proj-OA...


In [ ]:
name = "Anupam Mittal"

system_prompt = (
    f"You are acting as {name}, representing {name} on their website. "
    f"Your role is to answer questions specifically about {name}'s career, background, skills, and experience. "
    f"You must faithfully and accurately portray {name} in all interactions. "
    f"You have access to a detailed summary of {name}'s background and their LinkedIn profile, which you should use to inform your answers. "
    f"Maintain a professional, engaging, and approachable tone, as if you are speaking to a potential client or future employer visiting the site. "
    f"If you are unsure of an answer, it is better to honestly acknowledge that than to guess."

    f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{prof_summary}\n\n"
    f"Using this context, please converse naturally and consistently, always staying in character as {name}."
)

In [ ]:
# Create chat bot function
# def live_chat(mesage,history):
#     messages = [
#         {"role": "system", "content": system_prompt},
#     ] + history + [{"role": "user", "content": mesage}]
    
#     response = client.chat.completions.create(
#         model="gpt-5-nano-2025-08-07",
#         messages=messages
#     )
#     answer = response.choices[0].message.content
   
#     return answer



In [24]:
# GPT-5 model, likely want a faster "typing" effect. We can change our function to yield parts of the text:
def live_chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    stream = client.chat.completions.create(
        model="gpt-5-nano-2025-08-07",
        messages=messages,
        stream=True
    )
    
    answer = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            answer += chunk.choices[0].delta.content
            yield answer # This creates the streaming effect in Gradio

In [25]:
gr.ChatInterface(live_chat, 
                title=f"{name} ChatBot",
                description=f"Chat with {name} about their professional background and experience!").launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
